# More Complicated Invertible Logic

In this notebook, we will tackle the topic of more general solvers, including both:

1) Generalizing the Linear Programming Method to find more complicated primitives

2) Combining Invertible Logic Primitives into much more complicated invertible circuits

In [95]:
import pulp
import numpy as np

# Simple ease-of-use wrapper class
class booleanFunction():

    def __init__(self,num_args : int,executable : callable):

        self.num_args = num_args
        self.executable = executable

    def __call__(self,args : list) -> bool:
        return self.executable(args[:self.num_args])

# Simple Hamiltonian finder
def FindHamiltonian(boolfunc : booleanFunction, node_count : int) -> tuple[np.array, np.array]:

    problem = pulp.LpProblem('problem',pulp.LpMinimize)

    nn_1o2 = (node_count*(node_count-1))//2

    # Energy
    E = pulp.LpVariable('E',-nn_1o2**2,-1,'Continuous')
    # Biases
    h = pulp.LpVariable.dicts('h',([0],[i for i in range(node_count)]),-10,10,'Integer')
    # Weights
    j = pulp.LpVariable.dicts('j',([0],[i for i in range(nn_1o2)]),-10,10,'Integer')
   
    # Dummy summation variables
    H_h, H_j = (0,0)

    for i in range(2 ** node_count):
        args = [2*((i >> j)%2)-1 for j in range(node_count)]
        count = 0

        # Initialize variables
        H_hp = 0; H_jp = 0

        # Build out constraints line by line
        for idx,a in enumerate(args):
            H_hp += a * h[0][idx]
            
            for jdx in range(idx+1,len(args)):
                H_jp += args[jdx]*a*j[0][count]
                count += 1

        # Summed terms of final energy
        H_h = H_h + H_hp
        H_j = H_j + H_jp

        # Do you satisfy the boolean function?
        if boolfunc(args):
            problem += (-1)*H_hp-H_jp-E == 0
        else:
            problem += (-1)*H_hp-H_jp-E-1 >= 0

    # Define the energy
    problem += -H_h-H_j-E

    # Solve problem
    status = problem.solve()

    if status == -1:
        raise AssertionError("No optimal solution found!")

    # Initialize output and pack
    h_out = np.zeros(node_count)
    J_out = np.zeros((node_count, node_count))
    count = 0

    for idx in range(node_count):
        h_out[idx] = pulp.value(h[0][idx])

        for jdx in range(idx+1,node_count):
            J_out[idx][jdx] = pulp.value(j[0][count])
            count += 1

    J_out += J_out.T

    return (h_out, J_out)

To start with, we'll implement an `XOR` gate. It seems as though this should be representable with 3 nodes, however an auxiliary node is required to encode all of the information to make a functional XOR gate work. These additional p-bits that don't have a direct interpretation to our function are called "auxilliary bits", in the convention that we use here, we list p-bits in our Hamiltonians in the following order:

1) Input p-bits representing lines in
2) Output p-bits representing lines out
3) Auxiliary p-bits necessary for mathematical reasons

Let us define an XOR gate and find its Hamiltonian with 3 nodes - observe that it doesn't, but in fact throws an error!

In [96]:
def xor(ab : list) -> int:
    # assuming false -> -1
    # ---||--- true -> +1
    A = ab[0] > 0
    B = ab[1] > 0
    C = ab[2] > 0
    return A ^ B == C 

class XOR(booleanFunction):

    def __init__(self):
        super().__init__(3, xor)

func = XOR()

h, J = FindHamiltonian(func, 3)

print(J,h)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/thomas/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9645fc298d414bd1973ab5eeab428386-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/9645fc298d414bd1973ab5eeab428386-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 89 RHS
At line 98 BOUNDS
At line 113 ENDATA
Problem MODEL has 8 rows, 7 columns and 56 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



AssertionError: No optimal solution found!

This error is as we suspected, let's try XOR now with some auxiliary p-bits.

In [98]:
h, J = FindHamiltonian(func, 4)

print("J =\n",J,"\n\nh =\n",h)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/thomas/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f07b87502aac42c185a94916ee7f09c8-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/f07b87502aac42c185a94916ee7f09c8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 229 RHS
At line 246 BOUNDS
At line 269 ENDATA
Problem MODEL has 16 rows, 11 columns and 176 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



AssertionError: No optimal solution found!